In [1]:
import numpy as np
import copy
import scipy.io

In [2]:
mat = scipy.io.loadmat('/Users/nikitakocherin/Downloads/BlackBoxData.mat')
x = mat['x'][0]
y = mat['y'][0]
x = (x / np.linalg.norm(x))
y = (y / np.linalg.norm(y))

In [3]:
def generate_first_monom_dict(I, x):
    def generate_first_mon(bias, x):
        if bias == 0:
                return x
        base = np.zeros(len(x), dtype='complex')
        if bias < 0:
            for i in range(-bias, len(x)):
                base[i] = x[i + bias]
        elif bias > 0:
            for i in range(len(x) - bias):
                base[i] = x[i + bias]
        return base
    result = []
    for i in range(-I, I + 1):
        result.append(generate_first_mon(i, x))
    return result

In [4]:
def generate_monom_dict(I, D, x):
    def generate_mon(bias, degree, x):
        if bias == 0:
            return np.power(np.absolute(x), degree)
        base = np.zeros(len(x), dtype='complex')
        if bias < 0:
            for i in range(-bias, len(x)):
                base[i] = x[i + bias]
        elif bias > 0:
            for i in range(len(x) - bias):
                base[i] = x[i + bias]
        return np.power(np.absolute(base), degree)
    result = []
    for i in range(-I, I + 1):
        for j in range(D + 1):
            result.append(generate_mon(i, j, x))
    return result

In [5]:
def multiplicate(count, monom_dict):
    if count == 1:
        for i in monom_dict:
            yield i
    elif count == 2:
        for i in monom_dict:
            for j in monom_dict:
                yield i * j
    elif count == 3:
        for i in monom_dict:
            for j in monom_dict:
                for k in monom_dict:
                    yield i * j * k

In [6]:
def generate_real_dict_generator_subset(I, D, M, x, num_sub, divider):
    monom_dict = generate_monom_dict(I, D, x)
    first_mon = generate_first_monom_dict(I, x)
    counter = 0
    for i in first_mon:
        to_mult = multiplicate(M, monom_dict)
        for j in to_mult:
            if counter % divider == num_sub:
                k = i * j
                k = k / np.linalg.norm(k)
                counter += 1
                yield k    
            else:
                counter += 1

In [7]:
def approximate_without_saving(x, y, I, D, M, divider, flag=True):
    u_n = y
    iter = 0
    v_arr = []
    vector_norm = []
    norm_u_n_prev = 0
    u_n_prev = copy.deepcopy(u_n)
    while(np.linalg.norm(u_n) > 0.01 and iter < 1000):
        vector_norm.append(np.linalg.norm(u_n))
        if abs(np.linalg.norm(u_n) - norm_u_n_prev) < 0.0000001:
            break
        norm_u_n_prev = np.linalg.norm(u_n)
        for j in range(divider):
            v_n1 = None
            val = 0
            my_dict = generate_real_dict_generator_subset(I, D, M, x, j, divider)
            for i in my_dict:
                s = np.absolute(np.vdot(u_n, i))
                nor = abs(s)
                if nor > val:
                    v_n1 = i
                    val = nor
            h_n1_wave = copy.deepcopy(v_n1)
            if flag:
                for i in v_arr:
                    #print("iter", iter)
                    #print(np.vdot(v_n1, i), np.vdot(i, v_n1))
                    h_n1_wave = h_n1_wave - np.vdot(i, v_n1) * i
            h_n1 = h_n1_wave / np.linalg.norm(h_n1_wave)
            v_arr.append(h_n1)
            u_n = u_n - np.vdot(h_n1, u_n) * h_n1 #поменял
        v_arr = []
        iter += 1
        if iter % 2 == 0:
            print(np.linalg.norm(u_n))
    print(np.linalg.norm(u_n), "iter:", iter)
    return vector_norm

In [11]:
%%time
u_norm_50 = approximate_without_saving(x, y, 2, 3, 2, 50)

0.0476220698415659
0.044944967511703855
0.043100061323473465
0.04205407623387158
0.04082610414559615
0.04003373048429378
0.039043008526355616
0.03804414196054878
0.03702210244454533
0.03668326412857028
0.03627741922755659
0.03599757364783698
0.0358050918028657
0.03567132302600407
0.03546010917595189
0.03536455125894848
0.03523489913895548
0.03508174719756233
0.0349975442095803
0.03446425302223117
0.0343288988093346
0.03416873089655853
0.0341053463043764


KeyboardInterrupt: 

In [12]:
%%time
u_norm_70 = approximate_without_saving(x, y, 2, 3, 2, 70)

KeyboardInterrupt: 